##### Question Section
Questions to study ->
Can I do half the data under one model and the remaining half of the data under another model and then ensemble the data?

For example, Age will be a U-curve, so using polynomial modelling for it is accurate, 
where as, we have seen fare to be linear, so using MLR for linear 
and using Random forest for Categorical. This would yield best results?? What kind of limitations does this idea have ?

## Steps :

1. Select relevent columns
1. Make Dummy Variables (OHE)
1. train validation split (since we already have test set)
1. Modelling
1. Tune model using GridsearchCV
1. Test ensambles

Select from the following Models:
1. decision trees
1. Naive Bayers
1. Linear Discriminant Analysis
1. K-nearest Neighbour
1. Logistic Regression
1. Neural Networks 
1. Support Vector Machin



In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### Importing Clean Data

In [2]:
train = pd.read_csv("train_clean.csv")
test =  pd.read_csv("test_clean.csv")

#### 1. Selecting Relevent Columns

In [3]:
train.drop(['PassengerId','Unnamed: 0','Name','Ticket','Ticket_num','Age','Fare'],inplace=True,axis=1)
test.drop(['PassengerId','Unnamed: 0','Name','Ticket','Ticket_num','Age','Fare'],inplace=True,axis=1)

In [4]:
print(train.shape)
train.head()

(891, 10)


,Survived,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Ticket_prefix,Age_binned,Fare_binned
0,0,3,male,1,0,0,S,A/5,19-27,1-10
1,1,1,female,1,0,1,C,PC,37-45,71-80
2,1,3,female,0,0,0,S,STON/O2.,19-27,1-10
3,1,1,female,1,0,1,S,None,28-36,51-60
4,0,3,male,0,0,0,S,None,28-36,1-10


In [5]:
print(test.shape)
test.head()

(418, 9)


,Pclass,Sex,SibSp,Parch,Cabin,Embarked,Ticket_prefix,Age_binned,Fare_binned
0,3,male,0,0,1,S,None,28-36,1-10
1,3,female,1,0,1,C,None,46-54,1-10
2,2,male,0,0,1,S,None,55-63,1-10
3,3,male,0,0,1,S,None,19-27,1-10
4,3,female,1,1,1,S,None,19-27,11-20


#### 2. Making Dummy Variables /OHE

See if I leave p class as labels of 1,2,3 the prediction will get be weight by the number that is meant for class representation.
[AAh! This is called natural ordered relationships. Explained in the medium article below] 

Things I wanna do 
1. One Hot: Passenger Class 
1. One Hot: Sex
1. One Hot: Spouse and sibling 
1. One Hot: Parent and Child
1. One Hot: Embarkment
1. Cabin detail (Present/Absent)
1. Embarked
1. Ticket classification based on pre text
1. Fare Binning ?
1. Age Binning ? (5 width)

Thing I need to read on is label encoding vs one hot encoding impact on category prediction ML model.
<a href='https://medium.com/@michaeldelsole/what-is-one-hot-encoding-and-how-to-do-it-f0ae272f1179'>A good Medium Article on this.</a>


In [6]:
train.columns

Index(['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked',
       'Ticket_prefix', 'Age_binned', 'Fare_binned'],
      dtype='object')

In [7]:
train.drop(['Ticket_prefix'],inplace=True,axis=1)
test.drop(['Ticket_prefix'],inplace=True,axis=1)

In [8]:
train_dum = pd.get_dummies(train)
test_dum = pd.get_dummies(test)

In [9]:
for i in train_dum.columns:
    if i in test_dum.columns:
        pass
    else : print(i)

Survived


In [10]:
test_dum.columns

Index(['Pclass', 'SibSp', 'Parch', 'Cabin', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Embarked_Unknown',
       'Age_binned_1-9', 'Age_binned_10-18', 'Age_binned_19-27',
       'Age_binned_28-36', 'Age_binned_37-45', 'Age_binned_46-54',
       'Age_binned_55-63', 'Age_binned_64-72', 'Age_binned_73-81',
       'Fare_binned_1-10', 'Fare_binned_100+', 'Fare_binned_11-20',
       'Fare_binned_21-30', 'Fare_binned_31-40', 'Fare_binned_41-50',
       'Fare_binned_51-60', 'Fare_binned_61-70', 'Fare_binned_71-80',
       'Fare_binned_81-90', 'Fare_binned_91-100'],
      dtype='object')

In [11]:
train_dum.columns 

Index(['Survived', 'Pclass', 'SibSp', 'Parch', 'Cabin', 'Sex_female',
       'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Embarked_Unknown', 'Age_binned_1-9', 'Age_binned_10-18',
       'Age_binned_19-27', 'Age_binned_28-36', 'Age_binned_37-45',
       'Age_binned_46-54', 'Age_binned_55-63', 'Age_binned_64-72',
       'Age_binned_73-81', 'Fare_binned_1-10', 'Fare_binned_100+',
       'Fare_binned_11-20', 'Fare_binned_21-30', 'Fare_binned_31-40',
       'Fare_binned_41-50', 'Fare_binned_51-60', 'Fare_binned_61-70',
       'Fare_binned_71-80', 'Fare_binned_81-90', 'Fare_binned_91-100'],
      dtype='object')

In [12]:
train_dum.shape

test_dum.shape

(418, 30)

#### 3. Train - Validation Splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_dum.drop(['Survived'],axis=1)
y = train_dum['Survived'].values

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=np.random)

#### 4. Modelling

In [ ]:
from sklearn import metrics

__Decision Tree Classifier__

In [ ]:
from sklearn.tree import DecisionTreeClassifier

depths=10
mean_acc=np.zeros((depths))

for i in range(1,depths+1):
    TitanicTree = DecisionTreeClassifier(criterion="entropy", max_depth = i)
    TitanicTree.fit(X_train,y_train)

    yhat = TitanicTree.predict(X_valid)
    mean_acc[i-1] = metrics.accuracy_score(y_valid, yhat)

mean_acc

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

In [ ]:
TitanicTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)

__KNN - Classifier__

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
Ks = 1 #Kstart
Ke = 15 #Kstop
mean_acc = np.zeros((Ke-Ks+1))

for n in range(Ks,Ke+1):
      
    KNN = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=KNN.predict(X_valid)
    mean_acc[n-1] = metrics.accuracy_score(y_valid, yhat)

mean_acc

In [ ]:
plt.plot(range(Ks,Ke+1),mean_acc,'g')
# plt.fill_between(range(1,Ks+1),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
# plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

Highest accuracy for 10 Neighbours

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 10).fit(X_train,y_train)

__Logistic Regression__


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score

LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
yhat = LR.predict(X_valid)
yhat_prob = LR.predict_proba(X_valid)

jaccard_similarity_score(y_valid, yhat)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_valid, yhat_prob)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid, yhat, average='weighted') 

__Support Vector Machines__

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf').fit(X_train, y_train) 

In [ ]:
yhat = clf.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_valid, yhat, average='weighted') 

In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_valid, yhat)

In [ ]:
metrics.accuracy_score(y_valid,yhat)

__Random Forest Classifier__

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

In [ ]:
yhat = rf.predict(X_valid)

In [ ]:
metrics.accuracy_score(y_valid, yhat)

#### 5. Comparing RMSE of each model using "train" data

In [ ]:
model_accuracy={'accuracy_training':{},'accuracy_valid':{}}
models = [TitanicTree,KNN,LR,clf,rf]

In [ ]:
TitanicTree.fit(X_train,y_train)
yhat = TitanicTree.predict(X_valid)
model_accuracy['accuracy_training']['DecisionTree']=cross_val_score(TitanicTree,X_train,y_train,scoring='accuracy',cv=3).mean()
model_accuracy['accuracy_valid']['DecisionTree']=metrics.accuracy_score(y_valid, yhat)

In [ ]:
KNN.fit(X_train,y_train)
yhat = KNN.predict(X_valid)
model_accuracy['accuracy_training']['KNN']=cross_val_score(KNN,X_train,y_train,scoring='accuracy',cv=3).mean()
model_accuracy['accuracy_valid']['KNN']=metrics.accuracy_score(y_valid, yhat)

In [ ]:
LR.fit(X_train,y_train)
yhat = LR.predict(X_valid)
model_accuracy['accuracy_training']['Logistic R']=cross_val_score(LR,X_train,y_train,scoring='accuracy',cv=3).mean()
model_accuracy['accuracy_valid']['Logistic R']=metrics.accuracy_score(y_valid, yhat)

In [ ]:
clf.fit(X_train,y_train)
yhat = clf.predict(X_valid)
model_accuracy['accuracy_training']['Support Vector M']=cross_val_score(clf,X_train,y_train,scoring='accuracy',cv=3).mean()
model_accuracy['accuracy_valid']['Support Vector M']=metrics.accuracy_score(y_valid, yhat)

In [ ]:
rf.fit(X_train,y_train)
yhat = rf.predict(X_valid)
model_accuracy['accuracy_training']['Random Forest']=cross_val_score(rf,X_train,y_train,scoring='accuracy',cv=3).mean()
model_accuracy['accuracy_valid']['Random Forest']=metrics.accuracy_score(y_valid, yhat)

In [ ]:
pd.DataFrame(model_accuracy)

#### 6. Feature Relevence

In [ ]:
import matplotlib.pyplot as plt
importance = TitanicTree.feature_importances_

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
# selecting features with importance >0
# All relevant features -> [:,[0,1,3,5,33,54,61,66,70,76]]
# removing feature with importance < .02
X_train_2 = X_train.iloc[:,[0,1,3,5,54,61,76]]
X_valid_2 = X_valid.iloc[:,[0,1,3,5,54,61,76]]
# test_2 = test_dum.iloc[:,[0,1,3,5,54,61,76]]

In [ ]:
TitanicTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)

In [ ]:
TitanicTree.fit(X_train_2,y_train)
yhat = TitanicTree.predict(X_valid_2)
metrics.accuracy_score(y_valid, yhat)